In [1]:
%pip install pytest

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import yaml
import logging
import sys

# Move two directories up from the current file location
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..", "..")))

# Now import the LoggerManager
from Research_docs.utils.my_logger_module import LoggerManager

In [3]:
import logging
import os
import time
import pytest
from pathlib import Path
from datetime import datetime
# from logger_manager import LoggerManager  # Ensure this matches your module name

# Ensure test log directory is clean before running tests
TEST_CATEGORY = "test_category"
TEST_NOTEBOOK = "test_logger"
logger = LoggerManager(TEST_NOTEBOOK, category=TEST_CATEGORY)

@pytest.fixture(scope="function", autouse=True)
def setup_and_cleanup():
    """Setup and cleanup before/after each test."""
    logger.close_logger()
    yield
    logger.close_logger()

# ---------------------------------------
# 1️⃣ Test Basic Logging Functionality
# ---------------------------------------
def test_basic_logging():
    logger.logger.info("Test log entry")
    log_files = list(logger.base_log_dir.glob("log_*.log"))
    assert len(log_files) > 0, "❌ Log file was not created!"

    with open(log_files[0], "r") as f:
        log_content = f.read()
        assert "Test log entry" in log_content, "❌ Log entry not found in log file!"

    print("✅ test_basic_logging passed!")

# ---------------------------------------
# 2️⃣ Test Logging at Different Levels
# ---------------------------------------
def test_debug_logging():
    logger.set_log_level("DEBUG")
    logger.logger.debug("Debug level message")
    
    log_files = list(logger.base_log_dir.glob("log_*.log"))
    assert len(log_files) > 0, "❌ No log file found!"

    with open(log_files[0], "r") as f:
        log_content = f.read()
        assert "Debug level message" in log_content, "❌ Debug message not logged!"

    print("✅ test_debug_logging passed!")

def test_error_logging():
    logger.logger.error("This is an error message")
    
    log_files = list(logger.base_log_dir.glob("log_*.log"))
    with open(log_files[0], "r") as f:
        log_content = f.read()
        assert "This is an error message" in log_content, "❌ Error message not logged!"

    print("✅ test_error_logging passed!")

def test_info_logging():
    logger.logger.info("This is an info message")
    
    log_files = list(logger.base_log_dir.glob("log_*.log"))
    with open(log_files[0], "r") as f:
        log_content = f.read()
        assert "This is an info message" in log_content, "❌ Info message not logged!"

    print("✅ test_info_logging passed!")

# ---------------------------------------
# 3️⃣ Test Log Rotation
# ---------------------------------------
def test_log_rotation():
    logger = LoggerManager("test_rotation", max_bytes=100)  # Small size for quick testing
    for i in range(50):  
        logger.logger.info(f"Log entry {i}")

    rotated_logs = list(logger.base_log_dir.glob("log_*.log"))
    assert len(rotated_logs) > 1, "❌ Log rotation did not occur!"

    print("✅ test_log_rotation passed!")

# ---------------------------------------
# 4️⃣ Test Logger Closing
# ---------------------------------------
def test_logger_closure():
    logger.close_logger()
    assert not logger.logger.hasHandlers(), "❌ Logger handlers were not removed!"

    print("✅ test_logger_closure passed!")

# ---------------------------------------
# 5️⃣ Test Searching Logs for Keywords
# ---------------------------------------
def test_search_logs():
    logger.logger.info("Search Test: YOLO")
    matches = logger.search_logs("YOLO")
    assert len(matches) > 0, "❌ Log search did not find expected entry!"

    print("✅ test_search_logs passed!")

# ---------------------------------------
# 6️⃣ Test Old Log Cleanup
# ---------------------------------------
def test_log_cleanup():
    old_log = logger.base_log_dir / "log_2000-01-01_00-00-00.log"
    with open(old_log, "w") as f:
        f.write("Old log content")

    logger._clean_old_logs(days=1)
    time.sleep(2)  # Allow cleanup thread to process

    assert not old_log.exists(), "❌ Old log was not cleaned!"

    zip_files = list(logger.base_log_dir.glob("log_*.zip"))
    assert len(zip_files) > 0, "❌ Old log was not compressed!"

    print("✅ test_log_cleanup passed!")

# ---------------------------------------
# 7️⃣ Test Dynamic Log Level Change
# ---------------------------------------
def test_log_level_change():
    logger.set_log_level("DEBUG")
    assert logger.logger.level == logging.DEBUG, "❌ Log level did not change to DEBUG!"

    logger.set_log_level("ERROR")
    assert logger.logger.level == logging.ERROR, "❌ Log level did not change to ERROR!"

    print("✅ test_log_level_change passed!")

# ---------------------------------------
# 8️⃣ Test Console Logging Enable/Disable
# ---------------------------------------
def test_enable_disable_console_logging():
    logger.enable_console_logging(False)
    for handler in logger.logger.handlers:
        if isinstance(handler, logging.StreamHandler):
            assert handler.level == logging.CRITICAL, "❌ Console logging not disabled!"

    logger.enable_console_logging(True)
    for handler in logger.logger.handlers:
        if isinstance(handler, logging.StreamHandler):
            assert handler.level == logging.INFO, "❌ Console logging not enabled!"

    print("✅ test_enable_disable_console_logging passed!")

# ---------------------------------------
# 9️⃣ Test Logging to Database (If Enabled)
# ---------------------------------------
@pytest.mark.skip(reason="Database logging is not implemented yet.")
def test_log_to_db():
    logger.log_to_db("INFO", "Database log test")
    logger.cursor.execute("SELECT * FROM logs WHERE message = 'Database log test'")
    result = logger.cursor.fetchone()
    assert result is not None, "❌ Database logging failed!"

    print("✅ test_log_to_db passed!")

# ---------------------------------------
# 🔟 Test Email Alerts (Mocked)
# ---------------------------------------
@pytest.mark.skip(reason="Email alert test requires SMTP setup.")
def test_email_alert():
    logger = LoggerManager("test_email", email_alerts=True)
    assert any(isinstance(h, logging.StreamHandler) for h in logger.logger.handlers), \
        "❌ Email alert handler was not added!"

    print("✅ test_email_alert passed!")

# ---------------------------------------
# 🚀 Run All Tests
# ---------------------------------------
if __name__ == "__main__":
    test_basic_logging()
    test_debug_logging()
    test_error_logging()
    test_info_logging()
    # test_log_rotation()
    test_logger_closure()
    # test_search_logs()
    test_log_cleanup()
    test_log_level_change()
    # test_enable_disable_console_logging()
    print("\n🎉 All tests completed successfully! 🎉")


2025-02-15 22:23:01,133 - INFO - Test log entry
2025-02-15 22:23:01,135 - DEBUG - Debug level message
2025-02-15 22:23:01,135 - ERROR - This is an error message
2025-02-15 22:23:01,136 - INFO - This is an info message


✅ Logger handlers initialized. Log file: C:\Users\sathish\Downloads\FL_ModelForAV\Research_docs\logs\test_category\test_logger\log_2025-02-15_22-23-01.log
🧹 Checking for old logs to clean...
✅ Logging initialized for 'test_logger' in category 'test_category'. Logs saved in C:\Users\sathish\Downloads\FL_ModelForAV\Research_docs\logs\test_category\test_logger\log_2025-02-15_22-23-01.log
✅ test_basic_logging passed!
✅ Log level set to DEBUG
✅ test_debug_logging passed!
✅ test_error_logging passed!
✅ test_info_logging passed!
✅ Logger 'test_logger' closed successfully.
✅ test_logger_closure passed!
🧹 Checking for old logs to clean...
🗑️ Old log C:\Users\sathish\Downloads\FL_ModelForAV\Research_docs\logs\test_category\test_logger\log_2000-01-01_00-00-00.log compressed to C:\Users\sathish\Downloads\FL_ModelForAV\Research_docs\logs\test_category\test_logger\log_2000-01-01_00-00-00.zip
✅ test_log_cleanup passed!
✅ Log level set to DEBUG
✅ Log level set to ERROR
✅ test_log_level_change passed!
